In [1]:
import math, re, os
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# from kaggle_datasets import KaggleDatasets
from tensorflow import keras
from functools import partial
from sklearn.model_selection import train_test_split
print("Tensorflow version " + tf.__version__)

import tensorflow.keras.layers as L
import tensorflow.keras.backend as K

Tensorflow version 2.4.0


In [2]:
from sklearn.metrics import accuracy_score

In [3]:
plt.rcParams["figure.figsize"] = (12,12)

In [4]:
IMAGE_SIZE = (512, 512)
AUTOTUNE = tf.data.experimental.AUTOTUNE
CLASSES = ['0', '1', '2', '3', '4']
NUM_CLASSES = len(CLASSES)

In [5]:
from keras.backend import sigmoid

class SwishActivation(tf.keras.layers.Activation):
    
    def __init__(self, activation, **kwargs):
        super(SwishActivation, self).__init__(activation, **kwargs)
        self.__name__ = 'swish_act'

def swish_act(x, beta = 1):
    return (x * sigmoid(beta * x))



from keras.utils.generic_utils import get_custom_objects
# from tf.keras.layers import Activation
get_custom_objects().update({'swish_act': SwishActivation(swish_act)})

# from tensorflow.keras.experimental import CosineDecay
import efficientnet.keras as eff

# decay_steps = int(round(NUM_TRAINING_IMAGES/BATCH_SIZE))*EPOCHS
# cosine_decay = CosineDecay(initial_learning_rate=1e-4, decay_steps=decay_steps, alpha=0.3)


inputs = tf.keras.layers.Input(shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3))
#     x = data_augmentation_layers(inputs)
#     model = tf.keras.applications.EfficientNetB0(include_top=False, input_tensor=x, weights="imagenet")
model = eff.EfficientNetB0(include_top=False, input_tensor=inputs, weights=None)


# Freeze the pretrained weights
# model.trainable = False

# Rebuild top
x = tf.keras.layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
x = tf.keras.layers.BatchNormalization()(x)

top_dropout_rate = 0.2
x = tf.keras.layers.Dropout(top_dropout_rate, name="top_dropout")(x)

x = tf.keras.layers.Dense(1024, kernel_regularizer=tf.keras.regularizers.L2(l2=0.001))(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation(swish_act)(x)
x = tf.keras.layers.Dropout(top_dropout_rate)(x)

x = tf.keras.layers.Dense(512, kernel_regularizer=tf.keras.regularizers.L2(l2=0.001))(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation(swish_act)(x)
x = tf.keras.layers.Dropout(top_dropout_rate)(x)

outputs = tf.keras.layers.Dense(NUM_CLASSES, activation="softmax", name="pred")(x)

# Compile
model = tf.keras.Model(inputs, outputs, name="EfficientNet")

loss = tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.3)
#     loss = tf.keras.losses.CategoricalCrossentropy()

#     model.compile(loss=taylor_cross_entropy_loss(), optimizer=tf.keras.optimizers.Adam(cosine_decay), 
#               metrics=["accuracy"])

# model.compile(loss=loss, optimizer=tf.keras.optimizers.Adam(cosine_decay), 
#               metrics=["accuracy"])

In [6]:
model = tf.keras.models.load_model('EfficientNetB0_512_512_87919.h5')

In [7]:
model.summary()

Model: "EfficientNet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 512, 512, 3) 0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 256, 256, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 256, 256, 32) 128         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, 256, 256, 32) 0           stem_bn[0][0]                    
_______________________________________________________________________________________

# Train on Pseudolabels

In [8]:
df = pd.read_csv('restest.csv')

In [9]:
df['label'] = df['label'].astype('string')

In [10]:
df.head()

,image_id,label
0,../input/train_images/1000812911.jpg,3
1,../input/train_images/100533489.jpg,2
2,../input/train_images/1007700625.jpg,3
3,../input/train_images/1008284502.jpg,3
4,../input/train_images/1014433832.jpg,3


In [14]:
BATCH_SIZE = 8

datagen = tf.keras.preprocessing.image.ImageDataGenerator(rotation_range=90,
                                                          width_shift_range=0.2,
                                                          height_shift_range=0.2,
                                                          horizontal_flip=True,
                                                          vertical_flip=True)

train_datagen_flow = datagen.flow_from_dataframe(
    dataframe=df,
    directory=None,
    x_col='image_id',
    y_col='label',
    target_size=(512, 512),
    batch_size=BATCH_SIZE,
    subset='training',
    seed=12345)

Found 1539 validated image filenames belonging to 5 classes.


In [15]:
history = model.fit(train_datagen_flow,
          steps_per_epoch=1539//BATCH_SIZE,
          epochs=10)

Epoch 1/10
192/192 [==============================] - 77s 398ms/step - loss: 1.9196 - accuracy: 0.9419
Epoch 2/10
192/192 [==============================] - 75s 392ms/step - loss: 1.8908 - accuracy: 0.9432
Epoch 3/10
192/192 [==============================] - 77s 400ms/step - loss: 1.8506 - accuracy: 0.9641
Epoch 4/10
192/192 [==============================] - 77s 399ms/step - loss: 1.8244 - accuracy: 0.9628
Epoch 5/10
192/192 [==============================] - 76s 396ms/step - loss: 1.7872 - accuracy: 0.9745
Epoch 6/10
192/192 [==============================] - 79s 410ms/step - loss: 1.7714 - accuracy: 0.9673
Epoch 7/10
192/192 [==============================] - 76s 395ms/step - loss: 1.7508 - accuracy: 0.9673
Epoch 8/10
192/192 [==============================] - 77s 399ms/step - loss: 1.7283 - accuracy: 0.9700
Epoch 9/10
192/192 [==============================] - 76s 394ms/step - loss: 1.7043 - accuracy: 0.9680
Epoch 10/10
192/192 [==============================] - 78s 402ms/step - l

In [ ]:
# model.fit(train_datagen_flow,
#           steps_per_epoch=16047//8,
#           epochs=5)

In [16]:
model.save('EfficientNetb0_512x512_pseudolabel_train.h5')